In [1]:
import numpy as np

# Week 2, day 5 
Solves the Saint-Venant equations (1D problem) using different numerical fluxes.

## Functions

In [ ]:
def exact_sol(mcells, hL, hR, uL, uR, coL, coR, gravit, time, xmin, xmax):
    tol = 1.E-16
    niter = 100

    h = np.zeros(mcells + 1)
    u = np.zeros(mcells + 1)
    co = np.zeros(mcells + 1)

    # Compute celerity on left and right states
    cL = np.sqrt(gravit * hL)
    cR = np.sqrt(gravit * hR)

    Hcrit = (uR - uL) - 2 * (cL + cR)  # depth positivity condition

    # Screen size for plotting
    fig, ax = plt.subplots(figsize=(10, 6))

    dx = (xmax - xmin) / mcells
    x = np.linspace(xmin, xmax, mcells + 1)
    S = x / time

    if hL <= 0 or hR <= 0 or Hcrit >= 0:
        # Dry bed case
        if hL <= 0:
            shR = uR + cR
            stR = uR - 2 * cR

            for i in range(mcells + 1):
                if S[i] >= shR:
                    h[i] = hR
                    u[i] = uR
                elif S[i] >= stR:
                    u[i] = (uR - 2 * cR + 2 * S[i]) / 3
                    C = (-uR + 2 * cR + S[i]) / 3
                    h[i] = C**2 / gravit
                else:
                    h[i] = hL
                    u[i] = uL
        else:
            if hR <= 0:
                shL = uL - cL
                stL = uL + 2 * cL

                for i in range(mcells + 1):
                    if S[i] <= shL:
                        h[i] = hL
                        u[i] = uL
                    elif S[i] <= stL:
                        u[i] = (uL + 2 * cL + 2 * S[i]) / 3
                        C = (uL + 2 * cL - S[i]) / 3
                        h[i] = C**2 / gravit
                    else:
                        h[i] = hR
                        u[i] = uR
            else:
                shL = uL - cL
                ssL = uL + 2 * cL
                ssR = uR - 2 * cR
                shR = uR + cR

                for i in range(mcells + 1):
                    if S[i] <= shL:
                        h[i] = hL
                        u[i] = uL
                    elif shL < S[i] <= ssL:
                        u[i] = (uL + 2 * cL + 2 * S[i]) / 3
                        C = (uL + 2 * cL - S[i]) / 3
                        h[i] = C**2 / gravit
                    elif ssL < S[i] <= ssR:
                        h[i] = 0
                        u[i] = 0
                    elif ssR < S[i] <= shR:
                        u[i] = (uR - 2 * cR + 2 * S[i]) / 3
                        C = (-uR + 2 * cR + S[i]) / 3
                        h[i] = C**2 / gravit
                    else:
                        h[i] = hR
                        u[i] = uR
    else:
        # Wet bed case
        hS, uS = wetbed(hL, hR, uL, uR, cL, cR, gravit, niter, tol)
        cS = np.sqrt(gravit * hS)

        if hS >= hL:
            qL = np.sqrt((hS + hL) * hS / (2 * hL**2))
            sL = uL - cL * qL
            left = True
        else:
            shL = uL - cL
            stL = uS - cS
            left = False

        if hS >= hR:
            qR = np.sqrt((hS + hR) * hS / (2 * hR**2))
            sR = uR + cR * qR
            right = True
        else:
            shR = uR + cR
            stR = uS + cS
            right = False

        for i in range(mcells + 1):
            if S[i] < uS:
                co[i] = coL
                if left:
                    if S[i] < sL:
                        h[i] = hL
                        u[i] = uL
                    else:
                        h[i] = hS
                        u[i] = uS
                else:
                    if S[i] < shL:
                        h[i] = hL
                        u[i] = uL
                    elif S[i] < stL:
                        u[i] = (uL + 2 * cL + 2 * S[i]) / 3
                        C = (uL + 2 * cL - S[i]) / 3
                        h[i] = C**2 / gravit
                    else:
                        h[i] = hS
                        u[i] = uS
            else:
                co[i] = coR
                if right:
                    if S[i] > sR:
                        h[i] = hR
                        u[i] = uR
                    else:
                        h[i] = hS
                        u[i] = uS
                else:
                    if S[i] > shR:
                        h[i] = hR
                        u[i] = uR
                    elif S[i] > stR:
                        u[i] = (uR - 2 * cR + 2 * S[i]) / 3
                        C = (-uR + 2 * cR + S[i]) / 3
                        h[i] = C**2 / gravit
                    else:
                        h[i] = hS
                        u[i] = uS

    return h, u, co, x

def wetbed(hL, hR, uL, uR, cL, cR, gravit, niter, tol):
    # Placeholder function for wet bed case, replace with actual logic
    # Solve for hS, uS
    hS = (hL + hR) / 2  # Approximation, should be replaced
    uS = (uL + uR) / 2  # Approximation, should be replaced
    return hS, uS

def exact_sol_adv(mcells, hL, hR, uL, uR, coL, coR, gravit, time, xmin, xmax):
    tol = 1.E-12

    h = np.zeros(mcells + 1)
    u = np.zeros(mcells + 1)
    co = np.zeros(mcells + 1)

    # Compute ratios
    R_h = hR / hL
    R_U = uL / uR if abs(uR) > tol else 0

    if uL >= 0 and uR >= 0:
        # Solution in the star region
        hS = hR
        uS = uL * np.sqrt(hL / hR)
        if R_U > np.sqrt(R_h):  # Right wave is a shock wave
            sR = uR + uS  # Shock speed
            right = True
        else:  # Right wave is a rarefaction
            shR = 2 * uR
            stR = 2 * uS
            right = False

    elif uL < 0 and uR < 0:
        # Solution in the star region
        hS = hL
        uS = -abs(uR) * np.sqrt(hR / hL)
        if R_U < np.sqrt(R_h):  # Left wave is a shock wave
            sL = uL + uS  # Shock speed
            left = True
        else:  # Left wave is a rarefaction
            shL = 2 * uL
            stL = 2 * uS
            left = False

    elif uL > 0 and uR < 0:
        if abs(R_U) < np.sqrt(R_h):  # Left wave is a shock wave
            hS = hL
            uS = -abs(uR) * np.sqrt(hR / hL)
            sL = uL + uS  # Shock speed
            left = True
        else:  # Right wave is a shock wave
            hS = hR
            uS = uL * np.sqrt(hL / hR)
            sR = uR + uS  # Shock speed
            right = True

    elif uL < 0 and uR > 0:
        uS = 0  # Transonic rarefaction
        shL = 2 * uL
        stR = 2 * uR

    # Space discretization
    dx = (xmax - xmin) / mcells
    x = np.linspace(xmin, xmax, mcells + 1)
    S = x / time

    for i in range(mcells + 1):
        if uS < -tol:
            if left:  # Left shock
                if S[i] < sL:
                    h[i] = hL
                    u[i] = uL
                elif S[i] <= 0:
                    h[i] = hS
                    u[i] = uS
                else:
                    h[i] = hR
                    u[i] = uR
            else:  # Left rarefaction
                if S[i] < shL:
                    h[i] = hL
                    u[i] = uL
                elif S[i] < stL:
                    u[i] = S[i] / 2
                    h[i] = hL
                elif S[i] <= 0:
                    h[i] = hS
                    u[i] = uS
                else:
                    h[i] = hR
                    u[i] = uR

        elif uS > tol:
            if right:  # Right shock
                if S[i] > sR:
                    h[i] = hR
                    u[i] = uR
                elif S[i] >= 0:
                    h[i] = hS
                    u[i] = uS
                else:
                    h[i] = hL
                    u[i] = uL
            else:  # Right rarefaction
                if S[i] > shR:
                    h[i] = hR
                    u[i] = uR
                elif S[i] > stR:
                    u[i] = S[i] / 2
                    h[i] = hR
                elif S[i] >= 0:
                    h[i] = hS
                    u[i] = uS
                else:
                    h[i] = hL
                    u[i] = uL

        else:  # uS = 0 (sonic rarefaction)
            if S[i] < shL:
                h[i] = hL
                u[i] = uL
            elif S[i] > stR:
                h[i] = hR
                u[i] = uR
            else:
                u[i] = S[i] / 2
                h[i] = hL if S[i] < 0 else hR

    hexact = h
    uexact = u
    coexact = co

    return hexact, uexact, coexact, x


def exact_speeds_FULL(hL, hR, uL, uR, cL, cR, gravit):
    tol = 1.0E-12
    niter = 100

    # Depth positivity condition (Hcrit)
    Hcrit = (uR - uL) - 2 * (cL + cR)

    # Wet bed case
    hS, uS = wetbed(hL, hR, uL, uR, cL, cR, gravit, niter, tol)

    # Left wave is a shock or rarefaction
    if hS >= hL:
        qL = np.sqrt((hS + hL) * hS / (2 * hL**2))
        sL = uL - cL * qL
    else:
        sL = uL - cL

    # Right wave is a shock or rarefaction
    if hS >= hR:
        qR = np.sqrt((hS + hR) * hS / (2 * hR**2))
        sR = uR + cR * qR
    else:
        sR = uR + cR

    return sL, sR

def exact_speeds_PREX(hL, hR, uL, uR, gravit):
    tol = 1.0E-6
    niter = 50

    cL = np.sqrt(gravit * hL)
    cR = np.sqrt(gravit * hR)

    # Wet bed case (depth positivity condition)
    hS, qS = wetbed_PREX(hL, hR, uL, uR, cL, cR, gravit, niter, tol)

    # Left wave is a shock or rarefaction
    if hS >= hL:
        qL = np.sqrt(0.5 * (hS / hL + 1))
        sL = -cL * qL
    else:
        sL = -cL

    # Right wave is a shock or rarefaction
    if hS >= hR:
        qR = np.sqrt(0.5 * (hS / hR + 1))
        sR = cR * qR
    else:
        sR = cR
    return sL, sR

In [ ]:
def exrs_ADV(hL, hR, uL, uR):
    R_h = hR / hL
    R_U = uL / uR

    if uL >= 0 and uR >= 0:
        # Solution in the star region
        hstar = hR
        ustar = uL * np.sqrt(hL / hR)
    elif uL < 0 and uR < 0:
        # Solution in the star region
        hstar = hL
        ustar = -abs(uR) * np.sqrt(hR / hL)
    elif uL > 0 and uR < 0:
        if abs(R_U) < np.sqrt(R_h):  # Left wave is a shock wave
            # Solution in the star region
            hstar = hL
            ustar = -abs(uR) * np.sqrt(hR / hL)
        else:  # Right wave is a shock wave
            # Solution in the star region (Non funziona!!!)
            hstar = hR
            ustar = uL * np.sqrt(hL / hR)
    elif uL < 0 and uR > 0:  # Transonic rarefaction
        # Solution in the star region
        hstar = hL  # or hR; It works well with both choices
        ustar = 0

    return hstar, ustar


def exrs_FULL(hL, hR, uL, uR, cL, cR, gravit):
    tol = 1.0E-8
    niter = 100

    # Evaluate the flow depth and velocity in the star region using wetbed function
    hS, uS = wetbed(hL, hR, uL, uR, cL, cR, gravit, niter, tol)
    cS = np.sqrt(gravit * hS)
    
    if hS >= hL:  # Left wave is a shock wave
        qL = np.sqrt((hS + hL) * hS / (2 * hL ** 2))
        sL = uL - cL * qL
        left = 1
    else:  # Left wave is a rarefaction
        shL = uL - cL
        stL = uS - cS
        left = 0

    if hS >= hR:  # Right wave is a shock
        qR = np.sqrt((hS + hR) * hS / (2 * hR ** 2))
        sR = uR + cR * qR
        right = 1
    else:  # Right wave is a rarefaction
        shR = uR + cR
        stR = uS + cS
        right = 0

    S = 0  # Assuming S is the star region speed
    
    # Check left wave condition
    if S < uS:
        if left == 1:
            if S < sL:
                hstar = hL
                ustar = uL
            else:
                hstar = hS
                ustar = uS
        else:
            if S < shL:
                hstar = hL
                ustar = uL
            else:
                if S < stL:
                    ustar = (uL + 2 * cL + 2 * S) / 3
                    C = (uL + 2 * cL - S) / 3
                    hstar = C ** 2 / gravit
                else:
                    hstar = hS
                    ustar = uS
    # Check right wave condition
    else:
        if right == 1:
            if S > sR:
                hstar = hR
                ustar = uR
            else:
                hstar = hS
                ustar = uS
        else:
            if S > shR:
                hstar = hR
                ustar = uR
            else:
                if S > stR:
                    ustar = (uR - 2 * cR + 2 * S) / 3
                    C = (-uR + 2 * cR + S) / 3
                    hstar = C ** 2 / gravit
                else:
                    hstar = hS
                    ustar = uS

    return hstar, ustar

In [2]:
def fun_qk(hs, hk):
    y = hs / hk
    qk = np.sqrt(0.5 * (y**2 + y))
    return qk

def geofun(he, hk, ck, gravit):
    if he <= hk:  # wave is a rarefaction wave
        c = np.sqrt(gravit * he)
        f = 2 * (c - ck)
        fd = gravit / c
    else:  # wave is a shock
        ges = np.sqrt(0.5 * gravit * (he + hk) / (he * hk))
        f = (he - hk) * ges
        fd = ges - 0.25 * gravit * (he - hk) / (ges * he**2)
    
    return f, fd

def geofun_PREX(he, hk, ak, gravit):
    if he <= hk:  # wave is a rarefaction wave
        a = np.sqrt(gravit * he)
        f = (2 / 3) * (he * a - hk * ak)
        fd = a
    else:  # wave is a shock
        ges = np.sqrt(0.5 * gravit * (he + hk))
        f = (he - hk) * ges
        fd = np.sqrt(gravit / 8) * (3.0 * he + hk) / np.sqrt(he + hk)
    
    return f, fd

def hll(hL, hR, uL, uR, aL, aR, gravit):
    hstar = (((aL + aR) / 2 + (uL - uR) / 4) ** 2) / gravit
    
    if hstar > hL:
        qL = np.sqrt(0.5 * (hstar + hL) * hstar / (hL ** 2))
    else:
        qL = 1
    
    if hstar > hR:
        qR = np.sqrt(0.5 * (hstar + hR) * hstar / (hR ** 2))
    else:
        qR = 1
    
    SL = uL - aL * qL
    SR = uR + aR * qR
    
    return SL, SR

def two_rar_approx_PREX(hL, hR, qL, qR, gravit):
    tol = 1.0E-12
    cL = np.sqrt(gravit * hL)
    cR = np.sqrt(gravit * hR)
    
    # Two rarefaction solution as starting value
    hS2 = (0.75 / np.sqrt(gravit) * (qL - qR) + 0.5 * (hL ** 1.5 + hR ** 1.5)) ** 2
    hS = hS2 ** (1. / 3.)
    
    if hS < tol:
        hS = tol
    
    qS = 0.5 * (qL + qR) + np.sqrt(gravit) / 3 * (hL ** 1.5 - hR ** 1.5)
    
    if abs(qS) < tol:
        qS = np.sign(qS) * tol
    
    return hS, qS

def wetbed(hL, hR, uL, uR, cL, cR, gravit, niter, tol):
    # Use two rarefaction solution as starting value
    hS = (1 / gravit) * (0.5 * (cL + cR) - 0.25 * (uR - uL)) ** 2
    
    h0 = hS
    cha = 1
    
    for i in range(niter):
        fL, fLd = geofun(hS, hL, cL, gravit)
        fR, fRd = geofun(hS, hR, cR, gravit)
        
        hS = hS - (fL + fR + uR - uL) / (fLd + fRd)
        
        cha = abs(hS - h0) / (0.5 * (hS + h0))
        
        if cha < tol:
            break
        
        if hS < 0:
            hS = tol
        
        h0 = hS
    
    uS = 0.5 * (uL + uR) + 0.5 * (fR - fL)
    
    return hS, uS

## Solve the 1D SWE

In [ ]:
# Constants
SMALL = 1.E-12
a = 0  # Left boundary
b = 50  # Right boundary
gate = (a + b) / 2  # Gate position for the dambreak

# Condition: the gate must be within the domain [a, b]
if gate > b:
    raise ValueError("Error: gate > b")

# Discretization parameters
mcells = 200  # Number of cells for numerical simulations

# Flux choice
# 1 --> Godunov method with exact Riemann problem
# 2 --> Lax-Friedrichs
# 3 --> Lax-Wendroff
# 4 --> FORCE
# 5 --> HLLC
# 6 --> Flux-splitting (2020) UPWIND: this is our method!!!
iflux = 5

gravit = 9.8  # Gravity
ntmaxi = 100000  # Maximum number of time cycles
timeout = 6  # Final time for computation
cfl = 0.9  # Courant number (choose cfl <1 for stable solutions)

# Initial conditions for simulations
itest = 2  # Test case number

# Define initial conditions based on the test case
if itest == 1:  # Left rarefaction - Right shock
    gate = 10.0
    h_init_L = 1.0
    h_init_R = 0.1
    UL = 2.5
    UR = 0.0
    timeout = 7.0
elif itest == 2:  # Left rarefaction - Right shock
    gate = 25.0
    h_init_L = 1.0
    h_init_R = 1.0
    UL = -5.0
    UR = 5.0
    timeout = 2.5
elif itest == 3:  # Two-shocks
    gate = 20.0
    h_init_L = 1.0
    h_init_R = 0.0005
    UL = 0.0
    UR = 0.0
    timeout = 4.0
elif itest == 4:  # Two rarefactions
    gate = 30.0
    h_init_L = 0.005
    h_init_R = 1.0
    UL = 0.0
    UR = 0.0
    timeout = 4.0
elif itest == 5:  # Two rarefactions (Red sea)
    gate = 25.0
    h_init_L = 0.1
    h_init_R = 0.1
    UL = -3.0
    UR = 3.0
    timeout = 5.0

# Discretization parameters
dx = (b - a) / mcells  # Spatial step
igate = round((gate - a) / dx)  # Index at which the gate is positioned
x = np.arange(a, b + dx, dx)  # Interface coordinate
xc = x[:mcells] + dx / 2  # Cell center coordinate

# Set initial conditions
h = np.zeros(mcells)
Q = np.zeros(mcells)
h[:igate] = h_init_L
Q[:igate] = h_init_L * UL
h[igate:] = h_init_R
Q[igate:] = h_init_R * UR

flux = np.zeros((2, mcells + 1))
flux_ADV = np.zeros((2, mcells + 1))
flux_PREX = np.zeros((2, mcells + 1))

# Time marching procedure
time = 0
for t in range(1, ntmaxi + 1):
    # Compute eigenvalues
    lambda_ = np.zeros((2, mcells))
    lambda_[0, :] = Q / h - np.sqrt(gravit * h)
    lambda_[1, :] = Q / h + np.sqrt(gravit * h)

    # Compute maximum eigenvalue for the Courant condition
    lambdamax = np.max(np.abs(lambda_))
    
    # Set the time step dt
    dt = cfl * dx / lambdamax
    if t < 10:
        dt = dt * 0.2

    # Calculate numerical fluxes
    for i in range(1, mcells):
        hL = h[i - 1]
        hR = h[i]
        uL = Q[i - 1] / h[i - 1]
        uR = Q[i] / h[i]
        qL = Q[i - 1]
        qR = Q[i]
        aL = np.sqrt(gravit * hL)
        aR = np.sqrt(gravit * hR)

        if iflux == 1:  # Godunov
            hstar, ustar = exrs_FULL(hL, hR, uL, uR, aL, aR, gravit)
            flux[0, i] = hstar * ustar
            flux[1, i] = ustar**2 * hstar + 0.5 * gravit * hstar**2

        elif iflux == 2:  # Lax-Friedrichs
            FL = np.array([hL * uL, uL**2 * hL + 0.5 * gravit * hL**2])
            FR = np.array([hR * uR, uR**2 * hR + 0.5 * gravit * hR**2])

            flux[0, i] = 0.5 * (FL[0] + FR[0]) - 0.5 * dx / dt * (hR - hL)
            flux[1, i] = 0.5 * (FL[1] + FR[1]) - 0.5 * dx / dt * (hR * uR - hL * uL)

        elif iflux == 3:  # Lax-Wendroff
            FL = np.array([hL * uL, uL**2 * hL + 0.5 * gravit * hL**2])
            FR = np.array([hR * uR, uR**2 * hR + 0.5 * gravit * hR**2])

            hLW = 0.5 * (hL + hR) - 0.5 * dt / dx * (FR[0] - FL[0])
            QLW = 0.5 * (uL * hL + uR * hR) - 0.5 * dt / dx * (FR[1] - FL[1])
            uLW = QLW / hLW

            flux[0, i] = hLW * uLW
            flux[1, i] = uLW**2 * hLW + 0.5 * gravit * hLW**2

        elif iflux == 5:  # HLL Flux
            SL, SR = hll(hL, hR, uL, uR, aL, aR, gravit)
            ustar = 0.5 * (uL + uR) + aL - aR

            FL = np.array([hL * uL, uL**2 * hL + 0.5 * gravit * hL**2])
            FR = np.array([hR * uR, uR**2 * hR + 0.5 * gravit * hR**2])

            Fhhl = (SR * FL - SL * FR + SR * SL * (np.array([hR, hR * uR]) - np.array([hL, hL * uL]))) / (SR - SL)

            if SL >= 0:
                flux[:, i] = FL
            elif SR <= 0:
                flux[:, i] = FR
            else:
                flux[:, i] = Fhhl

    # Transmissive boundary conditions
    flux[:, 0] = flux[:, 1]  # Left boundary
    flux[:, mcells] = flux[:, mcells - 1]  # Right boundary

    # Update formula
    h = h - dt / dx * (flux[0, 1:mcells + 1] - flux[0, :mcells])
    Q = Q - dt / dx * (flux[1, 1:mcells + 1] - flux[1, :mcells])

    u = Q / h
    Fr = u / np.sqrt(gravit * h)

    hmax = np.max(h) * 1.2
    hmin = np.min(h) * 0.8

    # Plotting results
    if time >= timeout:
        umax = np.max(u) * 1.2
        umin = np.min(u)

        plt.subplot(2, 1, 1)
        plt.plot(xc, h, 'o')
        plt.title(f'Water height at time = {timeout} s')
        plt.xlabel('x')
        plt.ylabel('h')

        plt.subplot(2, 1, 2)
        plt.plot(xc, Q / h, 'o')
        plt.title(f'Velocity at time = {timeout} s')
        plt.xlabel('x')
        plt.ylabel('u')

        plt.show
